In [32]:
import pandas as pd
import numpy as np

from time import sleep
from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [33]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
raw_df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1HOUR,
                                      str(365) + ' days ago UTC'))

# First 6 column represent Time and OHLCV values
raw_df = raw_df.iloc[:, :6]
raw_df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
raw_df['Time'] = pd.to_datetime(raw_df['Time'], unit='ms')
for column in raw_df.columns:
    if column != 'Time':
        raw_df[column] = raw_df[column].astype(float)

# Set Time as index
raw_df.set_index('Time', inplace=True)
raw_df = raw_df.iloc[:-1]

# Generate Technical Indicators

In [34]:
def get_ta(raw_df):
    df = raw_df.copy()
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['SMA200'] = df['Close'].rolling(200).mean()
    df['RSI'] = momentum.rsi(df['Close'])
    df['MACD'] = trend.macd(df['Close'])
    df['MACD_sig'] = trend.macd_signal(df['Close'])
    df['MACD_diff'] = trend.macd_diff(df['Close'])
    df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
    df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
    df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
    df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])
    df.dropna(inplace=True)
    return df


In [35]:
df = get_ta(raw_df)

# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [36]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2023-08-06 16:00:00,29012.92,29089.99,29000.52,29089.31,473.53674,29074.4422,29214.25835,53.218078,-8.502923,-7.165182,-1.337741,0.001078,18.628678,21.597236,20.789797
2023-08-06 17:00:00,29089.30,29097.50,29052.43,29088.54,347.62193,29070.7274,29213.29105,53.099451,-5.429014,-6.817948,1.388934,0.001396,17.375282,20.057068,20.495500
2023-08-06 18:00:00,29088.55,29102.91,29074.03,29087.82,358.55729,29067.1762,29212.32010,52.980526,-3.016248,-6.057608,3.041360,0.002518,16.368464,19.116330,20.412819
2023-08-06 19:00:00,29087.82,29100.00,29076.00,29088.00,213.37531,29064.9360,29211.42430,53.008861,-1.077172,-5.061521,3.984349,0.002127,15.433561,18.346231,19.590490
2023-08-06 20:00:00,29088.00,29170.00,29076.69,29135.78,615.11358,29066.4014,29210.63315,59.914294,4.265833,-3.196050,7.461883,0.002325,16.293405,15.698353,27.590351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-28 03:00:00,68149.44,68149.45,67940.94,67940.95,287.53753,67865.1882,66817.27950,47.425085,78.094384,192.532251,-114.437867,-0.000074,31.873977,31.174684,19.881131
2024-07-28 04:00:00,67940.94,68024.18,67300.00,67338.00,750.45729,67871.6680,66819.31050,40.220758,2.995602,154.624921,-151.629319,-0.008656,32.009630,36.457736,18.049083
2024-07-28 05:00:00,67338.00,67500.00,67066.66,67422.00,765.53150,67879.2678,66821.95910,41.552841,-49.175758,113.864786,-163.040543,-0.007157,32.397726,37.432473,17.037352


In [37]:
def get_signal(df):

    #SMA Golden Cross & Death Cross
    sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
    sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
    df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

    # MACD Below 0 Cross
    macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
    macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
    df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

    # RSI Overbought & Oversold
    rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
    rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
    df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

    # ADX Trend Confirmation
    adx_buy_cond = (df['ADX'] > 25) & (df['+DI'] > df['-DI'])
    adx_sell_cond = (df['ADX'] > 25) & (df['-DI'] > df['+DI'])
    df['Signal_adx'] = np.where(adx_buy_cond, 1, np.where(adx_sell_cond, -1, 0))

    # Momentum Trade
    momentum_buy_cond = (df['Momentum'] > 0) & (df['Momentum'].shift(1) > 0) & (df['Momentum'].shift(2) > 0) & (df['Momentum'].shift(3) > 0) & (df['Momentum'].shift(4) > 0) \
                        & (df['Momentum'].shift(5) > 0) & (df['Momentum'].shift(6) > 0) & (df['Momentum'].shift(7) > 0) & (df['Momentum'].shift(8) > 0) & (df['Momentum'].shift(9) > 0)
    momentum_sell_cond = (df['Momentum'] < 0) & (df['Momentum'].shift(1) < 0) & (df['Momentum'].shift(2) < 0) & (df['Momentum'].shift(3) < 0) & (df['Momentum'].shift(4) < 0) \
                        & (df['Momentum'].shift(5) < 0) & (df['Momentum'].shift(6) < 0) & (df['Momentum'].shift(7) < 0) & (df['Momentum'].shift(8) < 0) & (df['Momentum'].shift(9) < 0)
    df['Signal_momentum'] = np.where(momentum_buy_cond, 1, np.where(momentum_sell_cond, -1, 0))

    df['Buy/Sell Consensus'] = df['Signal_sma'] + df['Signal_macd'] + df['Signal_rsi'] + df['Signal_adx'] + df['Signal_momentum']

    return df

In [38]:
df = get_signal(df)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-06 16:00:00,29012.92,29089.99,29000.52,29089.31,473.53674,29074.4422,29214.25835,53.218078,-8.502923,-7.165182,...,0.001078,18.628678,21.597236,20.789797,0,0,0,0,0,0
2023-08-06 17:00:00,29089.30,29097.50,29052.43,29088.54,347.62193,29070.7274,29213.29105,53.099451,-5.429014,-6.817948,...,0.001396,17.375282,20.057068,20.495500,0,1,0,0,0,1
2023-08-06 18:00:00,29088.55,29102.91,29074.03,29087.82,358.55729,29067.1762,29212.32010,52.980526,-3.016248,-6.057608,...,0.002518,16.368464,19.116330,20.412819,0,0,0,0,0,0
2023-08-06 19:00:00,29087.82,29100.00,29076.00,29088.00,213.37531,29064.9360,29211.42430,53.008861,-1.077172,-5.061521,...,0.002127,15.433561,18.346231,19.590490,0,0,0,0,0,0
2023-08-06 20:00:00,29088.00,29170.00,29076.69,29135.78,615.11358,29066.4014,29210.63315,59.914294,4.265833,-3.196050,...,0.002325,16.293405,15.698353,27.590351,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-28 03:00:00,68149.44,68149.45,67940.94,67940.95,287.53753,67865.1882,66817.27950,47.425085,78.094384,192.532251,...,-0.000074,31.873977,31.174684,19.881131,0,0,0,-1,0,-1
2024-07-28 04:00:00,67940.94,68024.18,67300.00,67338.00,750.45729,67871.6680,66819.31050,40.220758,2.995602,154.624921,...,-0.008656,32.009630,36.457736,18.049083,0,0,0,-1,0,-1
2024-07-28 05:00:00,67338.00,67500.00,67066.66,67422.00,765.53150,67879.2678,66821.95910,41.552841,-49.175758,113.864786,...,-0.007157,32.397726,37.432473,17.037352,0,0,0,-1,0,-1


# Run Test Trades on Historcical Data

In [39]:
historical_df = df[df['Buy/Sell Consensus'] != 0]
historical_df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-06 17:00:00,29089.30,29097.50,29052.43,29088.54,347.62193,29070.7274,29213.29105,53.099451,-5.429014,-6.817948,...,0.001396,17.375282,20.057068,20.495500,0,1,0,0,0,1
2023-08-07 07:00:00,29052.00,29110.57,29051.99,29094.09,756.16992,29077.1702,29199.31235,49.400517,11.385373,12.958755,...,0.001917,16.026643,21.226722,18.268513,0,-1,0,0,0,-1
2023-08-07 19:00:00,29094.42,29154.92,29050.78,29154.91,1490.61338,29075.7904,29183.63405,56.663360,-13.288978,-15.032902,...,0.004787,20.609721,24.890534,19.937247,0,1,0,0,0,1
2023-08-08 03:00:00,29173.03,29208.32,29173.03,29195.08,736.25201,29096.3440,29178.03315,56.079600,31.566027,19.078007,...,0.003730,15.529303,18.842252,22.602240,0,0,0,0,1,1
2023-08-08 04:00:00,29195.09,29233.00,29181.81,29225.63,727.22432,29099.4850,29177.80650,58.252371,34.740780,22.210562,...,0.004317,15.382845,18.051926,23.676448,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-28 03:00:00,68149.44,68149.45,67940.94,67940.95,287.53753,67865.1882,66817.27950,47.425085,78.094384,192.532251,...,-0.000074,31.873977,31.174684,19.881131,0,0,0,-1,0,-1
2024-07-28 04:00:00,67940.94,68024.18,67300.00,67338.00,750.45729,67871.6680,66819.31050,40.220758,2.995602,154.624921,...,-0.008656,32.009630,36.457736,18.049083,0,0,0,-1,0,-1
2024-07-28 05:00:00,67338.00,67500.00,67066.66,67422.00,765.53150,67879.2678,66821.95910,41.552841,-49.175758,113.864786,...,-0.007157,32.397726,37.432473,17.037352,0,0,0,-1,0,-1


In [40]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0

In [41]:
for idx, row in historical_df.iterrows():
    if (row['Buy/Sell Consensus'] >= 2) & (open_trade == False):
        coin = (balance / row['Close']) *0.999
        bought_price = row['Close']
        open_trade = True
    elif (row['Buy/Sell Consensus'] <= -2) & (open_trade == True):
        open_trade = False
        print(f"PnL:{(row['Close']-bought_price)/bought_price * 100}")
        balance = coin * row['Close'] *0.999
        coin = 0

PnL:-2.064272812910123
PnL:-0.5811474274025049
PnL:-0.43594123937550555
PnL:-0.961584349247738
PnL:-1.8281851075985587
PnL:0.8161880670702112
PnL:3.6446758906559706
PnL:-0.7801320206728518
PnL:26.66736781001343
PnL:-0.37111703639237975
PnL:4.501794165506847
PnL:-2.8569409914083215
PnL:-2.4712501615309947
PnL:-0.06560376239052165
PnL:14.67976366243981
PnL:-4.62065713265652
PnL:-1.4579873160448351
PnL:0.4515956924288818
PnL:-2.029125346527538
PnL:-1.6350823410144273
PnL:-2.805338941471094
PnL:2.5357173872174137
PnL:-2.7780660545973204
PnL:-1.3106624322889444
PnL:14.746034840622505
PnL:22.706302838763058
PnL:4.368010289088295
PnL:-6.624048820736145
PnL:5.6128839433511235
PnL:2.2684726920291087
PnL:-5.161179938646545
PnL:-2.7042173902549704
PnL:6.69416482115294
PnL:-3.2121000391632295
PnL:-2.052649122250086
PnL:3.238354576124288
PnL:-2.090006784881639
PnL:0.8087015743747492
PnL:-2.983511117454115
PnL:-1.4237920886208315
PnL:-1.6141278972252853
PnL:-0.32163742690058483
PnL:-3.86056130148591

In [42]:
balance

1501.8301514026277

In [43]:
1 - 1/1000

0.999

# Run Test Trades on Live Data

In [44]:
raw_df

,Open,High,Low,Close,Volume
Time,,,,,
2023-07-29 09:00:00,29295.44,29300.14,29265.73,29282.00,626.82238
2023-07-29 10:00:00,29282.00,29294.24,29260.81,29282.01,271.67292
2023-07-29 11:00:00,29282.01,29304.00,29267.15,29267.16,444.20990
2023-07-29 12:00:00,29267.16,29335.25,29261.87,29294.01,661.01850
2023-07-29 13:00:00,29294.00,29332.25,29289.29,29293.98,564.29946
...,...,...,...,...,...
2024-07-28 03:00:00,68149.44,68149.45,67940.94,67940.95,287.53753
2024-07-28 04:00:00,67940.94,68024.18,67300.00,67338.00,750.45729
2024-07-28 05:00:00,67338.00,67500.00,67066.66,67422.00,765.53150


In [45]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0
balance_df = {f"{raw_df.index[-1]}": balance}

In [46]:
while True:
        # Get new hour data
        hour_data = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                                Client.KLINE_INTERVAL_1HOUR,
                                        str(2) + ' hour ago UTC'))

        # First 6 column represent Time and OHLCV values
        hour_data = hour_data.iloc[:, :6]
        hour_data.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

        # Convert first column into datetime
        hour_data['Time'] = pd.to_datetime(hour_data['Time'], unit='ms')
        for column in hour_data.columns:
                if column != 'Time':
                        hour_data[column] = hour_data[column].astype(float)

        # Set Time as index
        hour_data.set_index('Time', inplace=True)

        # Join with old data
        raw_df.drop(raw_df.tail(1).index,inplace=True)
        raw_df = pd.concat([raw_df, hour_data])

        # Get TAs and Trading Signals
        df = get_ta(raw_df)
        df = get_signal(df)

        # Simulate Trade
        if (df.iloc[-2, -1] >= 2) & (open_trade == False):
                coin = (balance / df.iloc[-1,0]) *0.999
                balance = 0
                bought_price = df.iloc[-1,0]
                open_trade = True
        elif (df.iloc[-2, -1] <= -2) & (open_trade == True):
                open_trade = False
                print(f"PnL:{(df.iloc[-1,0]-bought_price)/bought_price * 100}")
                balance = coin * df.iloc[-1,0] *0.999
                coin = 0

        balance_df[f'{df.index[-1]}'] =  balance + coin*df.iloc[-1,0]
        print(f"Concencus: {df.iloc[-1,-1]}")
        print(f"Current Balance: {balance} USD and {coin} BTC at {df.index[-1]}")

        sleep(3600)
     

Concencus: -1
Current Balance: 1000 USD and 0 BTC at 2024-07-28 08:00:00
Concencus: -1
Current Balance: 1000 USD and 0 BTC at 2024-07-28 09:00:00
